In [1]:
import pandas as pd

In [2]:
# load the big file into memory
# again this is very memory consuming, and I think there is some better way to do it
df = pd.read_csv('steam_raw.csv')

In [3]:
df.head(10)

,Unnamed: 0,steamid,appid,voted_up,votes_up,votes_funny,weighted_vote_score,playtime_forever,playtime_at_review,num_games_owned,num_reviews,review,unix_timestamp_created,unix_timestamp_updated
0,0,76561198107294407,10,True,0,0,0.0,8549,8549,3,1,best gameplay ever created,1619038050,1619038050
1,1,76561198011733201,10,True,0,0,0.0,25381,25381,74,1,Timeless. Thank you volvo.,1619013754,1619013754
2,2,76561198168961276,10,True,0,0,0.0,681,616,26,6,this game literally made me love guns and fps ...,1619008681,1619008681
3,3,76561198957877160,10,True,0,0,0.0,2071,1990,5,1,If this gets a rework in 2021 it's going to be...,1619003706,1619003706
4,4,76561199050314447,10,True,0,0,0.0,36718,36718,3,1,ok,1618994842,1618994842
5,5,76561198888512591,10,True,0,0,0.0,1108,1108,24,11,"It's been a long day without you, my friend\nA...",1618988018,1618988018
6,6,76561199161043109,10,True,0,0,0.0,2346,1700,3,1,love it,1618985600,1618985600
7,7,76561199057736010,10,True,0,0,0.0,10149,10149,9,3,old,1618966033,1618966033
8,8,76561198800558753,10,True,0,0,0.0,1191,964,15,1,old is gold :),1618965435,1618965435
9,9,76561199010296112,10,True,0,0,0.0,145,51,6,7,"Counter Strike is a game that well, is Gold bu...",1618954290,1618954290


In [4]:
df.dtypes

Unnamed: 0                  int64
steamid                     int64
appid                       int64
voted_up                     bool
votes_up                    int64
votes_funny                 int64
weighted_vote_score       float64
playtime_forever            int64
playtime_at_review          int64
num_games_owned             int64
num_reviews                 int64
review                     object
unix_timestamp_created      int64
unix_timestamp_updated      int64
dtype: object

In [6]:
# turn the unix time into readable datetime
df['time']=pd.to_datetime(df['unix_timestamp_created'],unit='s')
df['time'].head(10)

0   2021-04-21 20:47:30
1   2021-04-21 14:02:34
2   2021-04-21 12:38:01
3   2021-04-21 11:15:06
4   2021-04-21 08:47:22
5   2021-04-21 06:53:38
6   2021-04-21 06:13:20
7   2021-04-21 00:47:13
8   2021-04-21 00:37:15
9   2021-04-20 21:31:30
Name: time, dtype: datetime64[ns]

In [8]:
df['time'].max()

Timestamp('2021-04-23 04:35:33')

In [9]:
# Grouping all the comments by month
counter = df.groupby([pd.Grouper(key="time", freq="1M", origin='2011-01-01'),pd.Grouper('appid')])

In [10]:
# generating the monthly review number
review = counter.count()['review']

In [11]:
# generating the average playtime of user
# there is a version for playtime in last two weeks in another dataset, and I think that is way better than playtime at review, but this dataset has more game
playtime_at_review = counter.mean()['playtime_at_review']

In [12]:
# generating the monthly upvote rate for each game
voted_up = counter.mean()['voted_up']

In [13]:
# putting everything into a dataframe
df_flat = pd.DataFrame({'review':review, 'playtime': playtime_at_review, 'voted_up': voted_up})
df_flat

review      playtime  voted_up
time       appid                                  
2010-10-31 20            1    437.000000  1.000000
           70            1    339.000000  1.000000
           80            1   7992.000000  1.000000
           240           3  14632.666667  1.000000
           300           1   1684.000000  1.000000
...                    ...           ...       ...
2021-04-30 1031480       5   1951.800000  1.000000
           1032430      43    364.604651  0.744186
           1035610       5    235.200000  0.800000
           1046030      91    862.505495  0.923077
           1051310       3    242.000000  0.666667

[377790 rows x 3 columns]

In [15]:
# save the dataframe to a csv file
df_flat.to_csv('steam_per_month_review.csv')